### 11. 프로젝트:멋진 작사가 만들기

In [1]:
import re
import numpy as np
import tensorflow as tf
import glob
import os
from sklearn.model_selection import train_test_split

#### 1. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME') + '/Aiffel/11_lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print('데이터 크기:', len(raw_corpus))
print('Examples:\n', raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Hey, Vietnam, Vietnam, Vietnam, Vietnam', 'Vietnam, Vietnam, Vietnam Yesterday I got a letter from my friend', 'Fighting in Vietnam']


#### 2. 데이터 정제   

- 문장 부호 양쪽에 공백 추가
- 모든 문자들을 소문자로 변환
- 특수 문자 모두 제거

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

In [4]:
corpus = []
for sentence in raw_corpus:
    corpus.append(preprocess_sentence(sentence))
    
corpus[:10]

['<start> hey , vietnam , vietnam , vietnam , vietnam <end>',
 '<start> vietnam , vietnam , vietnam yesterday i got a letter from my friend <end>',
 '<start> fighting in vietnam <end>',
 '<start> and this is what he had to say <end>',
 '<start> tell all my friends that i ll be coming home soon <end>',
 '<start> my time it ll be up some time in june <end>',
 '<start> don t forget , he said to tell my sweet mary <end>',
 '<start> her golden lips as sweet as cherries and it came from <end>',
 '<start> vietnam , vietnam , vietnam , vietnam <end>',
 '<start> vietnam , vietnam , vietnam it was just the next day his mother got a telegram <end>']

#### 3. 평가 데이터셋 분리

- train data(80%), validation data(20%)

In [5]:
def tokenize(corpus):
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000,
    filters='',
    oov_token='<unk>'
    )
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  135    4 ...    0    0    0]
 [   2 1353    4 ...   13  308    3]
 [   2 1046   14 ...    0    0    0]
 ...
 [   5   34   45 ... 1160  143    3]
 [   2    3    0 ...    0    0    0]
 [   2    3    0 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0ae3df1350>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2  135    4 1353    4 1353    4 1353    4 1353    3    0    0    0]
[ 135    4 1353    4 1353    4 1353    4 1353    3    0    0    0    0]


In [8]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=2020)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (149670, 14)
Target Train: (149670, 14)


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

#### 4. 모델 만들기

In [20]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x):
        out = self.embedding(x) 
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [21]:
optimizer =  tf.keras.optimizers.Adam()
loss =  tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=7, validation_data=(enc_val, dec_val))

Epoch 1/7
584/584 [==============================] - 261s 447ms/step - loss: 3.3038 - val_loss: 2.9007
Epoch 2/7
584/584 [==============================] - 268s 459ms/step - loss: 2.7133 - val_loss: 2.6210
Epoch 3/7
584/584 [==============================] - 265s 454ms/step - loss: 2.3755 - val_loss: 2.4387
Epoch 4/7
584/584 [==============================] - 251s 429ms/step - loss: 2.0571 - val_loss: 2.3124
Epoch 5/7
584/584 [==============================] - 251s 429ms/step - loss: 1.7623 - val_loss: 2.2239
Epoch 6/7
584/584 [==============================] - 254s 435ms/step - loss: 1.5028 - val_loss: 2.1748
Epoch 7/7
584/584 [==============================] - 261s 447ms/step - loss: 1.2869 - val_loss: 2.1507


#### 5. 평가하기

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '

#### 6. 고찰

- 하이퍼 파라미터 값을 변경시켜가며 학습을 수행한 결과 embedding_size = 512, hidden_size = 2048로 하였을때 좋은 결과가 나왔고 8epoch 부터는 val_score값이 커지는 것을 관찰할 수 있어 7epoch만큼 학습을 시키주었습니다. 
- 텍스트는 전처리를 하는 과정이 복잡하고 이 또한 언어에 대한 지식이 높다면 좀 더 효과적인 전처리를 수행할 수 있을 것 같습니다.
- LSTM을 사용하는 경우 좋은 효과를 얻을 수는 있지만 학습에 상당한 시간이 소모되어 많은 테스트를 수행하기에는 어려움이 있었습니다. 
- 모델이 만드는 문장을 보았을 때 가사를 이용하여 학습을 시켜 너무 간단하거나 비유적인 문장이 생성되는 경우를 볼 수 있었습니다.
- 이 후 소설등의 데이터를 이용하여 비슷한 문체의 글을 생성할 수 있는지를 테스트해보면서 자연어 처리에 대한 공부를 하면 좋을 것 같습니다. 